In [1]:
%load_ext autoreload
%autoreload 2
%load_ext heat

In [51]:
import ipas
import matplotlib.pyplot as plt
import numpy as np
from functools import partial
import time
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
import dask
from mpl_toolkits.mplot3d import Axes3D
from shapely.ops import nearest_points
from pyquaternion import Quaternion
import copy as cp
from scipy import spatial 
from shapely.geometry import Point
from shapely.ops import nearest_points
import pandas as pd
from dask import dataframe as dd
import pickle
import scipy.stats as st

In [58]:
cluster = SLURMCluster(
queue='kratos',
walltime='04-23:00:00',
cores=1,
memory='20000MiB', #1 GiB = 1,024 MiB
processes=1)

cluster.scale(28)
client = Client(cluster)
print(client)

<Client: 'tcp://169.226.65.97:38127' processes=0 threads=0, memory=0 B>


In [64]:
client

Client Scheduler: tcp://169.226.65.97:38127 Dashboard: http://169.226.65.97:8787/status,Cluster Workers: 12 Cores: 12 Memory: 251.64 GB


In [1]:
def main():
    output = np.empty((len(phioarr),len(reqarr)),dtype=object)
    for phi in range(len(phioarr)):
        for r in range(len(reqarr)):
            output[phi,r] = dask.delayed(ipas.collect_clusters)(phioarr[phi], reqarr[r], nclusters, ncrystals,rand_orient)
            #ipas.collect_clusters(phioarr[phi], reqarr[r], nclusters, ncrystals,rand_orient)
#     delayeds = client.compute(delayeds)
#     output = client.gather(delayeds)
    
    return output

In [69]:
def compute():
    agg_as = np.empty((len(phioarr),len(reqarr), nclusters))
    agg_bs = np.empty((len(phioarr),len(reqarr), nclusters))
    agg_cs = np.empty((len(phioarr),len(reqarr), nclusters))
    phi2Ds = np.empty((len(phioarr),len(reqarr), nclusters))
    cplxs = np.empty((len(phioarr),len(reqarr), nclusters))
    dds = np.empty((len(phioarr),len(reqarr), nclusters))
    
    gather = client.compute([*output.tolist()])  #only parallelizing agg r bins
    gather = client.gather(gather)

    gather = np.array(gather)
    print(np.shape(gather))
    agg_as = gather[:,:,0,:]
    agg_bs = gather[:,:,1,:]
    agg_cs = gather[:,:,2,:]
    phi2Ds = gather[:,:,3,:]
    cplxs = gather[:,:,4,:] 
    dds = gather[:,:,5,:]

    print('DONE!')
    return agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds

In [75]:
if __name__ == '__main__':
    phioarr=np.logspace(-2, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)

    reqarr = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000]
    #reqarr = [1]
    nclusters = 300         #changes how many aggregates per aspect ratio to consider
    ncrystals = 2
    rand_orient = True    #randomly orient the seed crystal and new crystal: uses first random orientation
    
    output = main()
    agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds = compute()

(20, 28, 6, 300)
DONE!


In [102]:
results = {'agg_as': agg_as, 'agg_bs':agg_bs, 'agg_cs':agg_cs, 'phi2Ds':phi2Ds, \
               'cplxs':cplxs, 'dd':dd}

In [77]:
np.shape(results['agg_as'])

NameError: name 'results' is not defined

In [76]:
#import bz2
#sfile = bz2.BZ2File('smallerfile', 'w')
with open('../instance_files/instance_iceice_rand_rall', "ab") as f:
    for data in agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds:
        pickle.dump(data, f)

In [78]:
f = open('../instance_files/instance_iceice_rand_rall', 'rb')
agg_as = pickle.load(f)
agg_bs = pickle.load(f)
agg_cs = pickle.load(f)
phi2Ds = pickle.load(f)
cplxs = pickle.load(f)
dds = pickle.load(f)
f.close()

In [82]:
class IceClusterBatch():
    """A collection of IceCluster objects."""
    
    def __init__(self, nclusters, rxs, rys, rzs, phi2Ds, cplxs, dds, numaspectratios, numrs):

        self.nclusters=nclusters
        self.rxs = rxs
        self.rys = rys
        self.rzs = rzs
        self.phi2Ds = phi2Ds
        self.cplxs = cplxs
        self.dds = dds
        self.numaspectratios = numaspectratios
        self.numrs = numrs

    def calculate_error(self, data, ch):
        mean = np.mean(data)
        std = np.std(data)
        shape = (mean/std)**2
        scale = (std**2)/mean
        shapech = mean/(self.numaspectratios*ch)

        pos_error = mean + std
        neg_error = mean - std

        min_data = min(data)
        max_data = max(data)

        return(pos_error, neg_error, min_data, max_data, mean)        


    def fit_distribution(self, data, normed = True, facecolor='navy', alpha=1.0,**kwargs):

        """Model data by finding best fit distribution to data"""
        # Get histogram of original data

        #fig = plt.figure(figsize=(5,7))
        #ax = plt.subplot(131)

        data = np.array(data)
        data[np.isinf(data)] = min(data)
        data[np.isnan(data)] = min(data)
        if np.isinf(data).any():
            print('inf True')
        y, x = np.histogram(data, density=True)
        #print('x hist',x)
        xx = (x + np.roll(x, -1))[:-1] / 2.0

        
        params = st.gamma.fit(data)
        arg = params[:-2]
        #fig = plt.figure(figsize=(10,7))
        #ax = fig.add_subplot(111)
        #n, bins, patches = plt.hist(data, bins=bins, normed=True,
        #                            color='navy',range=(min(data), max(data)),**kwargs)
        #pdf = self.make_pdf(distribution, gamma_params)
        pdf = st.gamma.pdf(xx, loc=params[-2], scale=params[-1], *arg)
        indmax = np.argmax(pdf)  #FIRST index where the highest prob occurs
        gammach_var = x[indmax] #characteristic of the distribution
        #ax = plt.plot(xx, pdf, lw=5, color='darkorange')
        #plt.show()

        return gammach_var

In [85]:
#find characteristic of gamma distribution
numaspectratios=len(agg_as[:,0,0])
numrs=len(agg_bs[0,:,0])
batch = IceClusterBatch(nclusters, agg_as, agg_bs, agg_cs, phi2Ds, \
                        cplxs, dds, numaspectratios, numrs)

agg_cs_ch = np.empty((numaspectratios, numrs), dtype=np.float64) #major
agg_as_ch = np.empty((numaspectratios, numrs), dtype=np.float64) #minor
agg_as_mean = np.empty((numaspectratios, numrs), dtype=np.float64)
dds_ch = np.empty((numaspectratios, numrs), dtype=np.float64)

for i in range(numaspectratios):
    for r in range(numrs):
        for c, data in enumerate([agg_cs, agg_as, dds]):
            if c == 0:
                agg_cs_ch[i,r] = batch.fit_distribution(data[i,r,:])
            if c == 1:
                agg_as_ch[i,r] = batch.fit_distribution(data[i,r,:])
                _,_,_,_,agg_as_mean[i,r] = batch.calculate_error(data[i,r,:], agg_as_ch[i,r])
            if c == 2:
                dds_ch[i,r] = batch.fit_distribution(data[i,r,:])
                

In [92]:
#write to file for output as array:
with open("../lookup_tables/minorax_flat_ch.dat","w") as file1:
    with open("../lookup_tables/majorax_flat_ch.dat","w") as file2:
        with open("../lookup_tables/majorax_flat_mean.dat","w") as file3:
            with open("../lookup_tables/dd_flat_ch.dat","w") as file4:
                file1.write('Ice-Ice collection (creating the crystals from pt arrays not db) for the flat orientation. \n')
                file1.write('Characteristic values taken from the peak of a fitted \n'\
                            'gamma distribution from 300 aggregates. \n'\
                            'Minor axis taken as the smallest axis from the fit ellipsoid'\n)
                file2.write('Ice-Ice collection (creating the crystals from pt arrays not db) for the flat orientation. \n')
                file2.write('Characteristic values taken from the peak of a fitted \n'\
                            'gamma distribution from 300 aggregates. \n'\
                            'Major axis taken as the largest axis from the fit ellipsoid'\n)
                file3.write('Ice-Ice collection (creating the crystals from pt arrays not db) for the flat orientation. \n')
                file3.write('Characteristic values taken from the peak of a fitted \n'\
                            'gamma distribution from 300 aggregates. \n')
                file4.write('Ice-Ice collection (creating the crystals from pt arrays not db) for the flat orientation. \n')
                file4.write('Characteristic values taken from the peak of a fitted \n'\
                            'gamma distribution from 300 aggregates. \n'\
                            'Volume ratio of agg (Vagg/Vellipse) subtracted from volume ratio of monomer (not 1.0)\n')
                            
                            
                for i in range(len(phioarr)):
                    for r in range(len(reqarr)):
                        print(i,r)
                        file1.write('%.3f, %.2f, %.2f \n' %(phioarr[i], reqarr[r],  agg_cs_ch[i,r]))
                        file2.write('%.3f, %.2f, %.2f \n' %(phioarr[i], reqarr[r],  agg_as_ch[i,r]))
                        file3.write('%.3f, %.2f, %.2f \n' %(phioarr[i], reqarr[r],  agg_as_mean[i,r]))
                        file4.write('%.3f, %.2f, %.4f \n' %(phioarr[i], reqarr[r],  dds_ch[i,r]))

file1.close()
file2.close() 
file3.close()
file4.close()

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
5 25
5 26
5 27
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
6 22
6 23
6 24
6 25
6 26
6 27
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
7 17
7 18
7 19


In [ ]:
#old format
with open("lookup_tables/oldformat_minorax_flat_ch.dat","wb") as file1:
    with open("lookup_tables/oldformat_majorax_flat_ch.dat","wb") as file2:
        with open("lookup_tables/oldformat_majorax_flat_mean.dat","wb") as file3:
            with open("lookup_tables/oldformat_dd_flat_ch.dat","wb") as file4:
file1.write('%d\t' %([r for r in reqarr]))

for phi in phioarr:
    
    

In [171]:
df = pd.DataFrame(rzs_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/oldformat_iceice_minorax_flat_ch.dat', sep='\t')
df = pd.DataFrame(rxs_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/oldformat_iceice_majorax_flat_ch.dat', sep='\t')
df = pd.DataFrame(rzs_mean, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/oldformat_iceice_minorax_flat_mean.dat', sep='\t')
df = pd.DataFrame(dds_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/oldformat_iceice_dd_flat_ch.dat', sep='\t')

In [166]:
np.shape(rxs_ch)

(20, 28)